In [1]:
# Execution Environment for KLUE-BERT
# pip install tensorflow==2.7.0
# pip install transformers==4.21.0
# TensorFlow version: 2.7.0
# Transformers version: 4.21.0
import tensorflow as tf
import transformers
print(f"TensorFlow version: {tf.__version__}")
print(f"Transformers version: {transformers.__version__}")

# Print the current date and time in the format:
# "YYYY-MM-DD HH:MM:SS.microseconds"
import datetime
def print_current_datetime():
    datetime_string = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
    print("CDT(Current Date and Time):", datetime_string)

2023-03-15 16:30:58.056841: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-15 16:30:58.056870: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/gbike/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


TensorFlow version: 2.7.0
Transformers version: 4.21.0


In [ ]:
# 전처리 후 데이터셋 : 총 117,609개
'''
import pandas as pd

# Read Excel data
df = pd.read_excel("[데이터셋] CS 신고관리 코멘트생성용 @ 전처리.xlsx", sheet_name="전처리")

# Write CSV data
df.to_csv("[데이터셋] CS 신고관리 코멘트생성용 @ 전처리.txt", sep="\t", index=False)

# 파일에서 "\t" 을 "|" 변환 후 Prompt |[SEP]| Target Text [다음줄] 포맷으로 변경
'''

In [2]:
# GPTv2 모델 사용

import random
import numpy as np
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, AutoTokenizer
from timeit import default_timer

# Load pre-trained GPT2 model and tokenizer
model = TFGPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2", from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")

# Load the dataset of prompts and target texts
# Prompt : a sentence, phrase, or a set of keywords that
# act as a starting point for generating a response text
with open("dataset_for_cs_comment_generation.txt", "r", encoding="utf-8") as f:
    dataset = [line.strip().split("|[SEP]|") for line in f.readlines()]

# Shuffle the dataset
random.seed(1234)
random.shuffle(dataset)

print("데이터셋 총 샘플 개수:", len(dataset))
print(dataset[:2][:])

sample_prompt = []
sample_target = []
for prompt, target in dataset:
    sample_prompt.append(prompt)
    sample_target.append(target)
print(sample_prompt[:2])
print(sample_target[:2])

input_ids = []
output_ids = []
for prompt, target in dataset:
    # Encode the prompt and target as input and output sequences
    # Not using special tokens such as [CLS], [SEP], etc.
    prompt_encoded = tokenizer.encode(prompt, add_special_tokens=False)
    target_encoded = tokenizer.encode(target, add_special_tokens=False)
    # Add the encoded sequences to the input and output lists
    input_ids.append(prompt_encoded)
    output_ids.append(target_encoded)

max_seq_length = 0
for arr in input_ids:
    if len(arr) > max_seq_length:
        max_seq_length = len(arr)
print("데이터셋 프롬프트 최대 토큰 길이:", max_seq_length)
max_seq_length = 0
for arr in output_ids:
    if len(arr) > max_seq_length:
        max_seq_length = len(arr)
print("데이터셋 타겟문장 최대 토큰 길이:", max_seq_length)

2023-03-15 16:31:01.948964: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-15 16:31:01.948990: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-15 16:31:01.949008: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-255-120-161): /proc/driver/nvidia/version does not exist
2023-03-15 16:31:01.949221: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 16:31:01.962120: W tensorflow/python/util/util.cc:368] Sets are not currently consider

데이터셋 총 샘플 개수: 117609
데이터셋 프롬프트 최대 토큰 길이: 137
데이터셋 타겟문장 최대 토큰 길이: 278


In [3]:
# Convert the dataset to tokenized sequences
input_ids = []
output_ids = []

start = default_timer()

for prompt, target in dataset:
    # Encode the prompt and target as input and output sequences
    # Not using special tokens such as [CLS], [SEP], etc.
    prompt_encoded = tokenizer.encode(prompt, add_special_tokens=False)
    target_encoded = tokenizer.encode(target, add_special_tokens=False)
    input_seq = prompt_encoded
    output_seq = target_encoded
    # Truncate sequences that are too long
    # For GPT2, the default value for max_seq_length is 1024
    # Using the default value (1024) causes OOM error
    # max_seq_length = model.config.n_positions
    max_seq_length = 280 # 데이터셋 타겟문장 최대 토큰 길이 (참고)
    if len(input_seq) > max_seq_length:
        input_seq = input_seq[:max_seq_length]
    if len(output_seq) > max_seq_length:
        output_seq = output_seq[:max_seq_length]
    # Pad sequences that are too short
    # The default value for padding in GPT2 tokenizer is 0
    # Using tokenizer.pad_token_id value (None) causes error
    # because None str is not int64 type, which is required
    padding_length_input_seq = max_seq_length - len(input_seq)
    if ( padding_length_input_seq > 0 ):
        input_seq += [0] * padding_length_input_seq
    padding_length_output_seq = max_seq_length - len(output_seq)
    if ( padding_length_output_seq > 0 ):
        output_seq += [0] * padding_length_output_seq
    '''
    padding_length_input_seq = max_seq_length - len(input_seq)
    if ( padding_length_input_seq > 0 ):
        input_seq += [tokenizer.pad_token_id] * padding_length_input_seq
    padding_length_output_seq = max_seq_length - len(output_seq)
    if ( padding_length_output_seq > 0 ):
        output_seq += [tokenizer.pad_token_id] * padding_length_output_seq
    '''
    # Add the encoded sequences to the input and output lists
    input_ids.append(input_seq)
    output_ids.append(output_seq)

# Convert inputs and outputs to numpy arrays
# also change the type from object to int64
input_ids = np.array(input_ids).astype('int64') # Vectorized prompts
output_ids = np.array(output_ids).astype('int64') # Vectorized target texts

# Split data into training and validation sets
split_length = int(len(input_ids) * 0.9)
train_input_ids = input_ids[:split_length]
train_output_ids = output_ids[:split_length]
val_input_ids = input_ids[split_length:]
val_output_ids = output_ids[split_length:]
print("총 데이터셋 개수:", len(input_ids))
print("훈련 데이터셋 수:", split_length)
print("검증 데이터셋 수:", len(input_ids) - split_length)

end = default_timer()
print("Time duration(in seconds):", end - start)

# Temporary setting to print full numpy array
'''
with np.printoptions(threshold=np.inf):
    print(train_input_ids.shape)
    print(train_input_ids[:2])
    print(train_output_ids.shape)
    print(train_output_ids[:2])
'''

총 데이터셋 개수: 117609
훈련 데이터셋 수: 105848
검증 데이터셋 수: 11761
Time duration(in seconds): 31.875500281108543


'\nwith np.printoptions(threshold=np.inf):\n    print(train_input_ids.shape)\n    print(train_input_ids[:2])\n    print(train_output_ids.shape)\n    print(train_output_ids[:2])\n'

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

print_current_datetime()

print("[모델 학습 시간]")
start = default_timer()

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss)

# 모델 학습 시 성능이 개선되지 않는 횟수가 2회를 초과하면 학습을 멈춤
earlystop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 2)

# 모델 학습
history = model.fit( train_input_ids, train_output_ids,
          validation_data=(val_input_ids,val_output_ids),
          callbacks=[earlystop],
          batch_size = 4, # 약 12GB 메모리사용
          epochs = 10)

end = default_timer()
print("Time duration(in seconds):", end - start)

print_current_datetime()

# 모델 저장

model.save_pretrained("output/finetuned-kogpt2-cs-comment-generation")
tokenizer.save_pretrained("output/finetuned-kogpt2-cs-comment-generation")

In [ ]:
import matplotlib.pyplot as plt

print(history.history.keys())
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation val_loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
# 모델 로드 및 테스트

print("[모델 로딩 시간]")
start = default_timer()

loaded_model = TFGPT2LMHeadModel.from_pretrained("output/finetuned-kogpt2-cs-comment-generation")
loaded_tokenizer = AutoTokenizer.from_pretrained("output/finetuned-kogpt2-cs-comment-generation")

end = default_timer()
print("Time duration(in seconds):", end - start)

print("[문장 생성 시간]")
start = default_timer()

prompt = "환불 요청 안함 | 탑승 전 | 엑셀,바퀴 | 엑셀이 작동을 안해요"
input_ids = tokenizer.encode(prompt, return_tensors="tf")
output_ids = model.generate(input_ids=input_ids, max_length=280, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Prompt: " + prompt)
print("Generated text: " + generated_text)

end = default_timer()
print("Time duration(in seconds):", end - start)

In [4]:
# 에폭 한번 학습당 약 24시간(하루) 걸림 @ No GPU
# Normal fine-tuning steps are around 10000s
# and the number of epochs are around 10s
print_current_datetime()

print("[모델 학습 시간]")
start = default_timer()

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss)

history = model.fit(train_input_ids, train_output_ids,
          validation_data=(val_input_ids,val_output_ids),
          batch_size = 4, # 약 12GB 메모리사용
          epochs = 1)

end = default_timer()
print("Time duration(in seconds):", end - start)

print_current_datetime()

print("[문장 생성 시간]")
start = default_timer()

# Generate text from prompt :
# In text generation tasks, a 2-gram refers to a sequence of two consecutive words
# and early_stopping parameter specifies whether to stop generation as soon as
# all beam hypotheses (num_beams=5) have generated an end-of-sequence token
prompt = "환불 요청 안함 | 탑승 전 | 엑셀,바퀴 | 엑셀이 작동을 안해요"
input_ids = tokenizer.encode(prompt, return_tensors="tf")
output_ids = model.generate(input_ids=input_ids, max_length=280, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Prompt: " + prompt)
print("Generated text: " + generated_text)

end = default_timer()
print("Time duration(in seconds):", end - start)

# 모델 저장

model.save_pretrained("output/finetuned-kogpt2-cs-comment-generation-epoch1")
tokenizer.save_pretrained("output/finetuned-kogpt2-cs-comment-generation-epoch1")

CDT(Current Date and Time): 2023-03-15 16:32:10.822238
[모델 학습 시간]


/home/gbike/anaconda3/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py:377: FutureWarning: The old compute_loss method is deprecated as it conflicts with the Keras compute_loss method added in TF 2.8. If you want the original HF compute_loss, please call hf_compute_loss() instead. From TF versions >= 2.8, or Transformers versions >= 5, calling compute_loss() will get the Keras method instead.
  return py_builtins.overload_of(f)(*args)


26462/26462 [==============================] - 86261s 3s/step - loss: 0.5165 - val_loss: 0.4861
Time duration(in seconds): 86261.62590811285
CDT(Current Date and Time): 2023-03-16 16:29:52.448478
[문장 생성 시간]
Prompt: 환불 요청 안함 | 탑승 전 | 엑셀,바퀴 | 엑셀이 작동을 안해요
Generated text: 환불 요청 안함 | 탑승 전 | 엑셀,바퀴 | 엑셀이 작동을 안해요 수거/점검 진행하도록 하겠습니다. 감사합니다. 또한,기는팀 점검하였습니다. 확인하여 이용에드 않도록하겠 감사 접수 꼼꼼히 확인 진행할 있도록습니다. 더욱 하겠 빠른릴 수 노력 운영립니다.합니다.습니다.합니다.합니다. 감사 감사 하겠 감사 수합니다. 노력 감사하겠 감사습니다.습니다.으로 불편 기 드 서비스 해당하여으로습니다. 접수합니다. 꼼꼼히리지시 부탁드리해 보려 반납겠며, 후 점 불편을 종료실 죄하다는 고객송 다시 번 환 양 답 이용어진될 신고금셔 도와하 미니 시 늦변일 경우구역 가능지면 참고 영업되 주차 기기 카드다 소요신 결제 확인 감사팀습니다. 확인 확인습니다. 수습니다.팀 감사 접수 접수 감사드습니다.드 감사 확인으로합니다. 수 감사 점검 감사 꼼꼼히 감사으로 감사하여 감사 운영 감사 노력합니다.팀팀합니다.으로 확인합니다. 확인
Time duration(in seconds): 107.95419706590474


('output/finetuned-kogpt2-cs-comment-generation-epoch1/tokenizer_config.json',
 'output/finetuned-kogpt2-cs-comment-generation-epoch1/special_tokens_map.json',
 'output/finetuned-kogpt2-cs-comment-generation-epoch1/vocab.json',
 'output/finetuned-kogpt2-cs-comment-generation-epoch1/merges.txt',
 'output/finetuned-kogpt2-cs-comment-generation-epoch1/added_tokens.json',
 'output/finetuned-kogpt2-cs-comment-generation-epoch1/tokenizer.json')